# main thesis project -fALFF

In [1]:
import pandas as pd
import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset, TensorDataset
from torchvision import transforms
from sklearn.model_selection import KFold
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## start of classsification using fALFF

In [2]:
control_base_path = '/data/zmohaghegh/Downloads/control_falff/fALFF_ALFF/fALFF_Normalised_z/'
mdd_base_path = '/data/zmohaghegh/Downloads/MDD_falff/fALFF_Normalised_z/'

#define data path for control and MDD seprately

control_files_path = [control_base_path + f for f in os.listdir(control_base_path) if f.startswith('fALFF')]
mdd_files_path = [mdd_base_path + f for f in os.listdir(mdd_base_path) if f.startswith('fALFF')]
print(len(control_files_path), len(mdd_files_path))

811 781


### load data with nan 

In [3]:
#loading data seprately and set a label column for Control=0 and MDD=1

control_dataset_fALFF_zero = [[nib.load(c).get_fdata(),0] for c in control_files_path]
mdd_dataset_fALFF_zero = [[nib.load(m).get_fdata(),1] for m in mdd_files_path]

##### define numpy array for doing the zero nan

In [4]:
control_fALFF_zero_nan2= control_dataset_fALFF_zero
mdd_fALFF_zero_nan2 = mdd_dataset_fALFF_zero

print(mdd_fALFF_zero_nan2[0][0].shape)
print(len(mdd_fALFF_zero_nan2))
print(mdd_fALFF_zero_nan2[0][1])

mdd_fALFF_zero_nan3 = np.array(mdd_fALFF_zero_nan2)
control_fALFF_zero_nan3 =np.array(control_fALFF_zero_nan2)
print(control_fALFF_zero_nan2[0][0].shape)
print(len(control_fALFF_zero_nan2))

control_fALFF_zero_nan2[0][1]

(91, 109, 91)
781
1
(91, 109, 91)
811


/data/zmohaghegh/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/data/zmohaghegh/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


0

In [5]:
len(mdd_dataset_fALFF_zero)

781

In [6]:
len(control_dataset_fALFF_zero)

811

In [7]:
len(mdd_dataset_fALFF_zero[0])
len(control_dataset_fALFF_zero[0])

2

In [8]:
mdd_dataset_fALFF_zero[0][0].shape
control_dataset_fALFF_zero[0][0].shape

(91, 109, 91)

In [9]:
control_dataset_fALFF_zero[800][0][:,34,56]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  0.79285711,  0.38341027,  0.7084465 ,  0.07752176,
       -0.05522991,  0.23621455, -0.34578028, -0.43479103, -1.26370239,
        0.33605337,  1.18806696, -0.56419861,  0.28790414,  0.58104312,
        0.91181123,  0.57643491,  0.0270216 ,  1.27149177, -1.447119  ,
        1.33609557, -1.07177174, -0.02550264,  0.76837009,  0.17475575,
       -0.17505197, -0.3093361 , -1.25800562, -1.1337204 , -0.14657176,
        0.90695989,  1.63714087,  1.67899549,  0.29507214, -0.90126312,
        0.63817197, -1.19335771,  0.73683125,  0.6093775 ,  0.58331841,
       -0.83863473, -0.69378096,  1.27149355,  0.2894654 ,  1.47491312,
        2.22593808,  1.77908099,  1.85931027,  2.66383648,  2.32862401,
       -0.35544148, -1.25970018,  1.31814718,  0.97653753, -1.15

In [10]:
mdd_dataset_fALFF_zero[700][0][:,34,56]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  2.81553602,  0.6552543 ,  0.6718654 ,  1.2881608 ,
        1.98794246,  1.845644  ,  1.62484765,  1.23959112,  1.06100953,
        0.46731874,  0.37081704,  0.03591866,  0.99368274,  0.86924809,
        1.32316458, -0.84771317, -0.54154211,  0.0110574 ,  1.14759302,
        0.37972936,  0.85268015,  0.31535962,  1.24906075,  0.1162873 ,
        1.25974882, -0.32019141,  1.54673874,  0.81460953,  1.06063545,
        1.43340743,  1.15204477,  0.44803467, -1.22255886, -1.12389338,
        0.40134168, -1.26558554,  0.69545043,  1.57197809, -1.81120825,
       -1.73631978, -1.43275416, -1.69512367, -0.23725261, -1.60847461,
       -2.10287309,  0.8710357 ,  0.11449484, -0.11623833,  1.4695338 ,
        2.35465837, -1.44191825,  0.96613085,  0.09745965, -0.20

In [11]:
#example control
zero_nan_control_800= np.nan_to_num(control_dataset_fALFF_zero[800][0],copy=True)
zero_nan_control_800[:,34,56]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.79285711,  0.38341027,  0.7084465 ,  0.07752176,
       -0.05522991,  0.23621455, -0.34578028, -0.43479103, -1.26370239,
        0.33605337,  1.18806696, -0.56419861,  0.28790414,  0.58104312,
        0.91181123,  0.57643491,  0.0270216 ,  1.27149177, -1.447119  ,
        1.33609557, -1.07177174, -0.02550264,  0.76837009,  0.17475575,
       -0.17505197, -0.3093361 , -1.25800562, -1.1337204 , -0.14657176,
        0.90695989,  1.63714087,  1.67899549,  0.29507214, -0.90126312,
        0.63817197, -1.19335771,  0.73683125,  0.6093775 ,  0.58331841,
       -0.83863473, -0.69378096,  1.27149355,  0.2894654 ,  1.47491312,
        2.22593808,  1.77908099,  1.85931027,  2.66383648,  2.32862401,
       -0.35544148, -1.25970018,  1.31814718,  0.97653753, -1.15

In [12]:
#example mdd
zero_nan_mdd_700= np.nan_to_num(mdd_dataset_fALFF_zero[700][0],copy=True)
zero_nan_mdd_700[:,34,56]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  2.81553602,  0.6552543 ,  0.6718654 ,  1.2881608 ,
        1.98794246,  1.845644  ,  1.62484765,  1.23959112,  1.06100953,
        0.46731874,  0.37081704,  0.03591866,  0.99368274,  0.86924809,
        1.32316458, -0.84771317, -0.54154211,  0.0110574 ,  1.14759302,
        0.37972936,  0.85268015,  0.31535962,  1.24906075,  0.1162873 ,
        1.25974882, -0.32019141,  1.54673874,  0.81460953,  1.06063545,
        1.43340743,  1.15204477,  0.44803467, -1.22255886, -1.12389338,
        0.40134168, -1.26558554,  0.69545043,  1.57197809, -1.81120825,
       -1.73631978, -1.43275416, -1.69512367, -0.23725261, -1.60847461,
       -2.10287309,  0.8710357 ,  0.11449484, -0.11623833,  1.4695338 ,
        2.35465837, -1.44191825,  0.96613085,  0.09745965, -0.20

In [13]:
#zero_nan_control
print(control_fALFF_zero_nan3.shape)
print(control_fALFF_zero_nan3[0][0].shape)
print(control_fALFF_zero_nan3[0][1])
print(control_fALFF_zero_nan3[100][1])
print(control_fALFF_zero_nan3[0].shape)
print(type(control_fALFF_zero_nan3))


for i in range(len(control_dataset_fALFF_zero)):    
    control_fALFF_zero_nan3[i][0] =np.nan_to_num(control_dataset_fALFF_zero[i][0],copy=True)
    control_fALFF_zero_nan3[i][1] =np.nan_to_num(control_dataset_fALFF_zero[i][1],copy=True)

(811, 2)
(91, 109, 91)
0
0
(2,)
<class 'numpy.ndarray'>


In [14]:
print(control_fALFF_zero_nan3.shape)
print(control_fALFF_zero_nan3[0][0].shape)
print(control_fALFF_zero_nan3[0][1]) ## mdd label for control=0
print(control_fALFF_zero_nan3[810][1])
control_fALFF_zero_nan3[810][0][:,34,56]


(811, 2)
(91, 109, 91)
0
0


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        8.54095221e-01,  1.63226497e+00,  8.17795992e-01,  9.50136483e-01,
        4.11450684e-01, -1.09165692e+00,  5.68864763e-01,  1.33423610e-02,
        8.38708058e-02,  1.47008955e-01, -5.16210198e-01,  3.18717659e-01,
        9.11356986e-01,  4.62050170e-01,  8.42347145e-01,  6.87091887e-01,
        1.57751429e+00, -1.30070031e+00, -1.02044439e+00,  7.76194215e-01,
       -5.35993397e-01,  1.48952782e-01,  2.03351808e+00,  1.85308084e-01,
       -1.28918397e+00,  3.66357476e-01, -7.58638024e-01, -1.01682770e+00,
       -1.00189006e+00,  3.38434488e-01,  5.43135107e-01,  1.21917140e+00,
        7.15234458e-01, -3.36432713e-05, -3.67613494e-01,  1.58524299e+00,
        1.59939373e+00,  

In [15]:
#zero_nan_mdd

print(mdd_fALFF_zero_nan3.shape)
print(mdd_fALFF_zero_nan3[0][0].shape)
print(type(mdd_fALFF_zero_nan3))

for i in range(len(mdd_dataset_fALFF_zero)):    
    mdd_fALFF_zero_nan3[i][0] =np.nan_to_num(mdd_dataset_fALFF_zero[i][0],copy=True)
    mdd_fALFF_zero_nan3[i][1] =np.nan_to_num(mdd_dataset_fALFF_zero[i][1],copy=True)

mdd_fALFF_zero_nan3.shape

(781, 2)
(91, 109, 91)
<class 'numpy.ndarray'>


(781, 2)

In [16]:
print(mdd_fALFF_zero_nan3.shape)

mdd_fALFF_zero_nan3.shape
print(mdd_fALFF_zero_nan3[0][0].shape)
print(mdd_fALFF_zero_nan3[0].shape)
print(mdd_fALFF_zero_nan3[0][1])
mdd_fALFF_zero_nan3[710][0][:,34,56]

(781, 2)
(91, 109, 91)
(2,)
1


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -1.35622644, -0.40713361, -0.01371213,  0.77628934,
       -1.27482736,  1.1860497 ,  0.574938  ,  0.30892873,  1.47411656,
        1.01888108,  0.59245348, -0.03579801,  0.91579568,  0.78738749,
        1.59277833,  0.94882649,  0.33899343,  0.42292801, -0.16982096,
       -0.31009254,  0.35575098, -0.74997598,  0.63261026,  1.88919067,
        2.26162815,  1.5200696 ,  0.26807237, -0.39947766,  1.12354076,
       -1.37134945,  0.85698241,  0.20198709, -0.20639515,  1.16627228,
        2.38697624,  0.14920443,  0.67725903,  0.1856637 , -0.2536771 ,
       -0.30439115,  0.80092996, -0.13461579, -0.3299183 ,  1.3129667 ,
        0.69235116,  0.13360614,  1.05824077,  1.03701222,  2.11989617,
        2.43561625,  0.70925099, -0.55656517,  0.91527492,  1.98

### load and add 1 channel to dimension of data 'without nan'

In [17]:
# np.reshape(control_dataset[0][0], (1, 91, 109, 91))

control_fALFF_4d_zero_nan = [[np.reshape(c[0], (1, 91, 109, 91)), c[1]] for c in control_fALFF_zero_nan3]
mdd_fALFF_4d_zero_nan = [[np.reshape(m[0], (1, 91, 109, 91)), m[1]] for m in mdd_fALFF_zero_nan3]

In [18]:
print(mdd_fALFF_4d_zero_nan[700][0].shape)
mdd_fALFF_4d_zero_nan[700][1]

(1, 91, 109, 91)


1

In [19]:
print(control_fALFF_4d_zero_nan[800][0].shape)
control_fALFF_4d_zero_nan[800][1]

(1, 91, 109, 91)


0

In [20]:
len(mdd_fALFF_4d_zero_nan)

781

In [21]:
len(mdd_fALFF_4d_zero_nan[0])

2

### concat data for control and MDD without nan

In [22]:
dataset_ukbb_fALFF_zero_nan= ConcatDataset([control_fALFF_4d_zero_nan, mdd_fALFF_4d_zero_nan])

print(dataset_ukbb_fALFF_zero_nan[0][0].shape)
print(dataset_ukbb_fALFF_zero_nan[1500][1])
print(dataset_ukbb_fALFF_zero_nan[810][1])


dataset_ukbb_fALFF_zero_nan[1500][0][0,:,54,45]

#subject number=1500, mdd label,fALFF value for voxels along axial with y ,z dimention = 54,45

(1, 91, 109, 91)
1
0


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.26051483, -0.14298151, -0.71571755,  0.57755512,
       -1.31374657, -0.56984562, -1.30225861,  0.62331015,  0.5041917 ,
       -0.83847451,  0.96808773,  0.67395806,  0.42258695,  0.68993944,
        0.31827262,  0.02916557,  0.45467517,  0.33197761,  0.91665024,
       -0.90823931, -1.11290121,  1.57923055,  0.14622138,  0.32043791,
        1.72918117,  0.14077128,  0.99111116,  0.38565841,  0.7349574 ,
       -1.12849081, -2.68588185, -1.5026052 , -1.72727966,  0.34865782,
        1.290205  ,  0.65884262,  0.49757457, -0.23207611,  1.03446007,
        0.79974961,  1.06815457, -0.36284623,  0.56689322,  0.99398375,
        1.81708336,  1.41056597,  1.93723655,  0.75482833,  0.48027435,
        0.75945824,  1.08402658,  1.06377399,  0.25535056,  0.64601636,
        1.66061056,  1.16460681, -0.03491591,  1.90987241,  0.43

In [23]:
len(dataset_ukbb_fALFF_zero_nan)

1592

In [24]:
len(dataset_ukbb_fALFF_zero_nan[0])

2

In [25]:
print(dataset_ukbb_fALFF_zero_nan[800][1]) # subject 800 label MDD =0
print(dataset_ukbb_fALFF_zero_nan[1500][1]) # subject 1500 label MDD =1 

#problem solved : mdd_zero_nan label have problem : instead of 1 integer we have an array (91,109,91)

print(dataset_ukbb_fALFF_zero_nan[0][1])

dataset_ukbb_fALFF_zero_nan[1500][1].shape

0
1
0


()

In [26]:
dataset_ukbb_fALFF_zero_nan[1500][1]

1

### define Network architecture

In [27]:
#Define a Convolutional Neural Network : BASED ON https://www.biorxiv.org/content/10.1101/2019.12.17.879346v1

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.downsample = nn.AvgPool3d(2, stride=2, padding=0)
        
        self.CNNlayer = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, stride=1),
            nn.ELU(),
            nn.Conv3d(64, 16, kernel_size=3, stride=1),
            nn.ELU(),
            nn.MaxPool3d(2)
        )
        
        self.flat1 = nn.Linear(160000, 16)   
        self.flat2 = nn.Linear(16, 1)
            
    def forward(self, x):
        x=self.downsample(x)
        #print(f'avg-pool: {x.size()}\n----------')
        #print(f'number of nan in this layer = {torch.isnan(x).sum()}')
        
        x=self.CNNlayer(x)
        #print(f'convolution1+2+maxpool: {x.size()} \n----------')
        
        x=x.reshape(x.size(0), -1)
        #print(f'reshape after cnn: {x.size()}\n----------')
        
        x=F.elu(self.flat1(x))
        #print(f'fully-connected1: {x.size()}\n----------')
                    
        x=self.flat2(x)
        #print(f'fully-connected2: {x.size()}\n----------')
        
        return x

### lunch wandb

In [28]:
import wandb
!wandb login 390734ff44d817dbba59927d4eb542e564627b3b

wandb: Appending key for api.wandb.ai to your netrc file: /data/zmohaghegh/.netrc


In [29]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {
            'values': [8, 10, 20]
        },
        'batch_size': {
            'values': [10, 32, 64]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4]
        },
    }
}

In [30]:
sweep_id = wandb.sweep(sweep_config, project="classifier-falff-sweep-hyper-parameter")

Create sweep with ID: hfinfn2b
Sweep URL: https://wandb.ai/zahramhn/classifier-falff-sweep-hyper-parameter/sweeps/hfinfn2b


### Train and Test (without CV)

In [31]:
### spilit data to train-test and validation

dataset_size= len(dataset_ukbb_fALFF_zero_nan)
train_count = int(0.8 * dataset_size) 
test_count = dataset_size - train_count

print(dataset_size)
print(train_count)
print(test_count)

1592
1273
319


In [32]:
def train():
    
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
            'epochs': 10,
            'batch_size': 10,
            'learning_rate': 1e-3,
        }

    # Initialize a new wandb run
    wandb.init(project='classifier-falff-sweep-hyper-parameter',config=config_defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    #Define a Loss function 
    loss_function = nn.BCEWithLogitsLoss()  

    #load dataset 
    train_dataset, test_dataset = torch.utils.data.random_split(dataset_ukbb_fALFF_zero_nan, (train_count, test_count))
    train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)  
    test_dataset_loader  = torch.utils.data.DataLoader(test_dataset , batch_size=config.batch_size, shuffle=True)

    #### load data splits
    dataloaders = {'train': train_dataset_loader, 'test': test_dataset_loader}

    #define the newtwork 
    network = ConvNet()
    network = network.double()

    #create the optimizer
    optimizer = optim.SGD(network.parameters(), momentum=0.9, lr = config.learning_rate, weight_decay= 1e-3)

    for epoch in range(0, config.epochs):
        print(f'Starting epoch {epoch+1}')

        train_loss = 0
        total =0
        correct=0

        # train model/network 
        for i, data in enumerate(train_dataset_loader, 0):
            #print(f'train {i}')

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the gradient buffers
            optimizer.zero_grad()

            # forward pass
            outputs = network(inputs)
            outputss = outputs.squeeze(1) #### [10,1] ---> [10]

            # prediction
            predicted = outputss.data > 0.0
            
            labels = labels.double()

            #calcuate loss/error
            loss = loss_function(outputss, labels)

            # backward pass
            loss.backward()

            # Does the update , gradient descent
            optimizer.step() 

            #loss_list.append(loss.item())

            train_loss += loss.item()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'acc={ 100 * correct /total}')
        print(f'loss={train_loss /total}')

        wandb.log({ "epoch": epoch , "train_Loss_falff": train_loss /total, "train_acc_falff ": 100 * correct / total })

    print('Training process has finished')    

### sweep over hyper parameter : hyper parameter tuning 

In [34]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: zhovzbdh with config:
wandb: 	batch_size: 10
wandb: 	epochs: 8
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: zahramhn (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Starting epoch 1


/data/zmohaghegh/venv/lib64/python3.6/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


acc=48.07541241162608
loss=0.07063503991501946
Starting epoch 2
acc=52.94579732914376
loss=0.06935929620966498
Starting epoch 3
acc=55.223880597014926
loss=0.06874459467076718
Starting epoch 4
acc=57.42340926944226
loss=0.06782706298544165
Starting epoch 5
acc=56.08798114689709
loss=0.06687469144300773
Starting epoch 6
acc=67.16417910447761
loss=0.06505575174636807
Starting epoch 7
acc=68.34249803613511
loss=0.0636888785239809
Starting epoch 8
acc=71.72034564021995
loss=0.06188122702377794
Training process has finished


epoch,7
train_Loss_falff,0.06188
train_acc_falff,71.72035
_runtime,4136
_timestamp,1615508428
_step,7


...

epoch,9
train_Loss_falff,0.01073
train_acc_falff,65.04321
_runtime,9465
_timestamp,1615891716
_step,9


epoch,▁▂▃▃▄▅▆▆▇█
train_Loss_falff,█▇▆▆▅▄▃▂▂▁
train_acc_falff,▁▁▃▂▂▄▄▇██
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: q778c821 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Starting epoch 1
acc=50.58915946582875
loss=0.010970675019426435
Starting epoch 2
acc=51.846032992930084
loss=0.010846758673534126
Starting epoch 3
acc=55.14532600157109
loss=0.010704158797071586
Starting epoch 4
acc=60.172820109976435
loss=0.010525265576135822
Starting epoch 5
acc=54.59544383346426
loss=0.010708777491673977
Starting epoch 6


wandb: Ctrl + C detected. Stopping sweep.


In [35]:
list(network.parameters())
#wandb.watch(network)

NameError: name 'network' is not defined

In [ ]:
    loss_v = criterion(outputs_v, labels) 
        # Although the loss is one value, it represents the *average* of losses over all examples in the batc
    loss_v_list.append(loss_v.item()) # one item at the time

    total_v = labels.size(0) # get it in case the last batch has different number of sample
    correct_v = (predicted == labels).sum().item() # sum the correct answers for this batch
    acc_list_v.append(correct_v / total_v) # n of correct / n of sample in this batch

    total_ep += total_v # sum all the samples. it must end up being the tot training set
    correct_ep += correct_v # sum together the right answers in entire training set
    loss_ep += (loss_v.item()*len(labels))
            
# calculate metrices of this epoch
current_ep_loss=loss_ep/total_ep
            
current_ep_acc=(correct_ep/total_ep)
acc = accuracy_score(F1_labels, F1_pred)
balacc= balanced_accuracy_score(F1_labels, F1_pred)
F1_Score = f1_score(F1_labels, F1_pred, average='weighted')
tn, fp, fn, tp = confusion_matrix(F1_labels, F1_pred).ravel() 

In [ ]:
if not best_acc or best_acc < current_ep_acc:
    
    best_acc = current_ep_acc
    torch.save(model.state_dict(), MODEL_STORE_PATH + '/' + summaryMeasure + '_BEST_model'
                               + '_CV' + str(k+1)
                               + '.pt')